# 模式匹配

模式匹配，是检查对象是否满足特定的条件。从C#7开始引入后，后续版本不断完善，让这个功能越来越完善。巧妙应用这个特性，可以让匹配的代码更加优雅。

**什么时候用**

当对象比较复杂，而对象本身的类型又不能帮助我们判定时。
  
**在哪用**

* if statement，注意善用`is`关键字
* switch expression/statement

## 测试类

为了后续表述方便，我们在此定义几个测试用类。

* 类中使用了C#9的新特性：`init`
* 它们虽然都是表达形状的类，但没有共同的基类

In [ ]:
public class Square
{
    public double Side { get; init; }
}

public class Circle
{
    public double Radius { get; init; }
}

public class Rectangle
{
    public double Length { get; init; }
    public double Height { get; init; }
}

public class Triangle
{
    public double Base { get; init; }
    public double Height { get; init; }
}

## Patterns

C#中现在有很多pattern，它们各有各的作用，经组合后有巨大的威力。

### Constant patterns

Constant patterns是最基础的patterns，因此它们很少单独使用。

In [ ]:
var rect = new Rectangle{Height = 6, Length =4};
if (rect.Height is 0)
{
    Console.WriteLine("This rectangel has no height");
}
else
{
    Console.WriteLine("This rectangel has height");
}

This rectangel has height


Constant patterns在C#7中引入。与直接使用`==`不同的是，`==`可以被重载，而constant patterns不能。

### Null Pattern

Null pattern是constant patterns的一个特例。用于判断对象是否为null。

In [ ]:
rect = null;
if (rect is null)
{
    throw new Exception("rect is null");
}

### Type pattern

Type pattern可以在检查对象是否为某个类实例的同时，声明一个被检查类型的变量。

In [ ]:
var shape = new Square{Side = 5};
if (shape is Square sq)
{
    Console.WriteLine($"The area of the squre is {sq.Side * sq.Side}");
}

The area of the squre is 25


In [ ]:
// the following is the same, but a bit verbose.
var s = new Square{Side = 5};
if (s is Square)
{
    var sq = s as Square;
    Console.WriteLine($"The area of the squre is {sq.Side * sq.Side}");
}

The area of the squre is 25


### Property pattern

Property pattern可检查一个对象的属性，在c#8中引入。

In [ ]:
if (rect is {Height: 6})
{
    Console.WriteLine("Rec height is 6");
}

// notice this is same with:
if (rect.Height is 6)
{
    Console.WriteLine("Rec height is 6");
}

Rec height is 6


Rec height is 6


Property pattern看起来有些多余，而且与constant pattern相比，可读性也差一些。不过后面会看到，它与其他模式组合起来，可以发挥更大的作用。

### Rational pattern

Rational pattern用于比较，在c#9中引入。

In [ ]:
var shape = new Circle{Radius = 11};
if (shape is Circle {Radius: > 10})
{
    Console.WriteLine("This is big circle");
}

// without rational and property pattern, we will do:
if (shape is Circle cr)
{
    if (cr.Radius > 10)
        Console.WriteLine("This is big circle");
}

This is big circle


This is big circle


上面同时使用了property pattern和rational pattern。与不用pattern的代码相比，更简洁，意图也更清晰。

### Negate pattern

Negate pattern用于否定判断，使用关键字`not`，在c#9引入。

In [ ]:
if (shape is Circle {Radius: not 0})
{
    Console.WriteLine("Circle has radius");
}

if (shape is not null)
{
    Console.WriteLine("not null is so much elegant now");
}

Circle has radius


not null is so much elegant now


Negate pattern让非null的判断变得如此优雅。Love it!

### Conjunctive and Disjunctive pattern

千万不要被这个名字吓到。这两个pattern与条件判断中`&&`与`||`运算符的作用相似：用关键字`and`和`or`用于把判断组对。也在c#9中引入。

In [ ]:
if(shape is Circle {Radius: > 0 and < 100} circle)
{
    Console.WriteLine($"Good circle with radius: {circle.Radius}");
}

Good circle with radius: 11


使用时注意有一些限制：
* and cannot be placed between two type patterns (unless they are targeting interfaces)
* or can be placed between two type patterns but it doesn’t support capturing
* and cannot be placed in a property pattern without a relational one
* or can be placed in a property pattern without a relational one and supports capturing
* or cannot be used between two properties of the same object
* and cannot be used between two properties of the same object, but it is implicit

不要被这些规则吓到了，事实上，在实际使用时，是否合理通常都可以推测出来:

```csharp
if (shape is Square and Circle ); // this will not compile
if (shape is Square or Circle); // OK!
if (shape is Square or Circle smt); // this will not compile
if (shape is Square { Side: 0 and 1 }); // this will not compile
if (shape is Square { Side: 0 or 1 } sq); // OK!
if (shape is Rectangle { Height: 0 or Length: 0 }); // this will not compile
if (shape is Rectangle { Height: 0 } or Rectangle { Length: 0 }); // OK!
if (shape is Rectangle { Height: 0 and Length: 0 }); // this will not compile
if (shape is Rectangle { Height: 0, Length: 0 } re); // OK! equivalent to the pattern above
```

## var pattern

var pattern的引入，突破了pattern matching只能用于常量的限制。可以在找到match的同时，声明一个变量，然后在pattern matching之外使用这个变量。

In [ ]:
var mySquare = new Square{ Side = 10 };
if (mySquare is Square {Side: var side} sq && side % 2 == 0)
{
    Console.WriteLine("This is an even square");
}

This is an even square


## Tupple pattern

当分支选择依赖于多个变量时，使用tupple pattern就非常方便。让我们看看使用这个pattern可以把FizBuz写得多么简洁。

In [ ]:
public class Foo
{
    public static string FizzBuzz(int a)
    {
        return (a % 3 == 0, a % 5 ==0) switch
        {
            (true, false) => "Fizz",
            (false, true) => "Buzz",
            (true, true) => "FizzBuzz",
            _ => $"{a}"
        };
    }
}

Console.WriteLine(Foo.FizzBuzz(1));
Console.WriteLine(Foo.FizzBuzz(3));
Console.WriteLine(Foo.FizzBuzz(5));
Console.WriteLine(Foo.FizzBuzz(15));

1


Fizz


Buzz


FizzBuzz


## Positional pattern

当对象有析构器时，就可以使用Positional pattern。使用时，与tupple pattern有类似之处。

In [ ]:
public class RectangleWithDector
{
    public double Length{ get; init; }
    public double Height{ get; init; }

    public void Deconstruct(out double length, out double height)
    {
        length = Length;
        height = Height;
    }
}

var rectd = new RectangleWithDector{ Length = 10, Height = 5 };
if (rectd is (10, _))
{
    Console.WriteLine("This rec has length 10");
}

This rec has length 10


# Conclusion

与所有特性一样，在合适的地方使用它。事实上，pattern matching并没有让C#的功能增加，它只是让一些需要判断的情形语法更简洁，更重要的是，让代码意图更加明显。这也是判断是否使用它的条件：**如果pattern matching可以让代码意图更明显，就使用它**。